In [44]:
from collections import defaultdict

f = open("dataset_865450_5.txt","r")
BLOSUM = open("BLOSUM62.txt","r")

string1, string2 = (f.readline()).strip('\n'),(f.readline()).strip('\n')

scoring = defaultdict(dict)
amino_acids = [str(x) for x in next(BLOSUM).split()]
while line := BLOSUM.readline():
    line_scores = [str(x) for x in line.split()]
    for i in range(1, len(line_scores)):
        scoring[line_scores[0]].update({amino_acids[i - 1]: int(line_scores[i])})

INDEL_PENALTY = 1

def FittingAlignment(v, w, scoring, indel_penalty):
    n = len(v) + 1
    m = len(w) + 1
    Backtrack = [[" "]*m for i in range(n)]
    s = [[0]*(m) for i in range(n)]
    for i in range(1, n):
        s[i][0] = 0 #-indel_penalty * i
    for j in range(1, m):
        s[0][j] = 0 #-indel_penalty * j
        
    for i in range(1, n):
        for j in range(1, m):
            match = scoring[v[i-1]][w[j-1]]
            
            if j == 1:
                s[i][j] = max(s[i-1][j] - indel_penalty, s[i][j-1] - indel_penalty, s[i-1][j-1] + match, 0)
            else:
                s[i][j] = max(s[i-1][j] - indel_penalty, s[i][j-1] - indel_penalty, s[i-1][j-1] + match)
            
            if s[i][j] == 0:
                Backtrack[i][j] = "f"
            elif s[i][j] == s[i-1][j] - indel_penalty:
                Backtrack[i][j] = "↓"
            elif s[i][j] == s[i][j-1] - indel_penalty:
                Backtrack[i][j] = "→"
            elif s[i][j] == s[i-1][j-1] + match:
                Backtrack[i][j] = "↘"
    
    max_length = float('-inf')
    max_i, max_j, = 0, 0
    for i in range(n):
        for j in range(m):
            if s[i][j] >= s[max_i][max_j]:
                max_length = s[i][j]
                max_i, max_j = i, j 
                
    if max_j < len(w):
        max_j = len(w)
        max_length = s[max_i][max_j]
    
    aligned1 = OutputV(Backtrack, v, max_i, max_j)
    aligned2 = OutputW(Backtrack, w, max_i, max_j)
    if aligned1[0] == '-':
        max_length -= INDEL_PENALTY
    
    return max_length, aligned1, aligned2

def OutputV(backtrack, v, i, j):
    if i == 0:
        return ""
    if j == 0:
        return ""
    if backtrack[i][j] == "↓":
        return OutputV(backtrack, v, i - 1, j) + v[i - 1]
    elif backtrack[i][j] == "→":
        return OutputV(backtrack, v, i, j - 1) + "-"
    elif backtrack[i][j] == "↘":
        return OutputV(backtrack, v, i - 1, j - 1) + v[i - 1]
    else:
        return '-'

def OutputW(backtrack, w, i, j):
    if i == 0:
        return w[:j]
    if j == 0:
        return ""
    if backtrack[i][j] == "↓":
        return OutputW(backtrack, w, i - 1, j) + "-"
    elif backtrack[i][j] == "→":
        return OutputW(backtrack, w, i, j - 1) + w[j - 1]
    elif backtrack[i][j] == "↘":
        return OutputW(backtrack, w, i - 1, j - 1) + w[j - 1]
    else:
        return OutputW(backtrack, w, i, j - 1) + w[j - 1]
    
length, v, w = FittingAlignment(string1, string2, scoring, INDEL_PENALTY)
print(length)
print(v)
print(w)

f.close()

466
DCMPDTDRHRGPSTGQRRTHDVWLTPREPMQ-ASRHWNHY-EKNTHGSGAWGCMDFW------C---YV------RPGVMMVDFFAS-VGENLSGQMYEYAMTTPWLKYKTWE
DCMPDTDRHRGP--------DVWLTPREPMQH-SRHWN-WSEKNTHGSGAWGCMDFWLSFWYACTDCYVQNQCMYRPGVMMVDFFASE-GENLSGQMYEYGMTTPWLKYKTWE
